In [1]:
%load_ext autotime
import geopandas as gpd # Geospatial data operations
import rasterio as rio # Geospatial imagery manipulation
import rasterio.plot
import pandas as pd # Tabular data
import os
import re
import rapidfuzz # Fuzzy string matching
from tqdm.auto import tqdm # Progress bars
from tqdm.contrib.concurrent import thread_map, process_map # Parallel operations
import matplotlib # Plots
import matplotlib.pyplot as plt
import shapely # Polygon operations
#import solaris.tile as tile # Tile splitting
#import solaris.data.coco as coco
import contextlib
import io
import rasterio # Raster imagery operations
from rasterio.vrt import WarpedVRT
from rasterio import transform
from rasterio.merge import merge # Merging tiles into mosaics
from glob import glob # Finding files
from shapely.geometry import box # Bounding box operations
matplotlib.rcParams['figure.figsize'] = (20, 10)
tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
import platform
if platform.system() == "Windows":
  prefix = "Z:/"
else:
  prefix = "ressci201900060-RNC2-Coastal/"

## Match shapefiles to images

In [2]:
# Set this to False to rebuild filelist.txt and meta.csv
use_cache = True

In [3]:
# This cell will read a list of files from Nick/filelist.txt if it exists, or build a list of files and save it to that location
# This is useful for quickly loading the list of files without having to search the entire directory structure
# You will need to delete the file and rerun this cell if files are created, renamed or moved

filename = prefix + "Nick/filelist.txt"
if use_cache and os.path.isfile(filename):
    filelist = pd.read_csv(filename, header=None).iloc[:,0]
else:
    def find_files(root):
        return pd.Series(glob(prefix + root + "**/**", recursive=True)).str.replace(prefix, "")
    filelist = pd.concat(thread_map(find_files, ["Gabrielle", "MaxarImagery", "Retrolens", "SpatialData/Mosaics"]))
    if platform.system() == "Windows":
        filelist = filelist.str.replace("\\", "/", regex=False)
    filelist.to_csv(filename, index=False, header=False)
filelist

0                                                                                       Gabrielle/
1                                                                                 Gabrielle/Orders
2                                                                            Gabrielle/Orders/AOIs
3                                          Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023WGS84.sbn
4                                               Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023.sbx
                                                    ...                                           
248342    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L02_R00000062_C00000065.tfw
248343    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L03_R00000020_C00000022.tfw
248344    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000125_C00000131.tif
248345    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000127_C00000133.tif
248346    

In [4]:
def check_filename(filename):
    # This regex only matches shapefiles that contain something date-like in their names
    match = re.search(r'/Shorelines/.+\d{4}\w*.shp$', filename)
    return bool(match)

shapefiles = filelist[filelist.apply(check_filename)]
df = shapefiles.to_frame(name="filename")
df

,filename
29902,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_18FEB2023.shp
29903,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_24DEC2022.shp
29906,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/NewChums_18FEB2023.shp
29907,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_24DEC2022.shp
29914,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_18FEB2023.shp
...,...
192152,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp
192157,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp
193261,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp
193272,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_Wellington_13FEB2021.shp


In [5]:
images = filelist[filelist.str.contains("Stack", case=False) & filelist.str.endswith((".jpg", ".jp2", ".tif", ".TIF"))]
images

32633     MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_2.tif
32634     MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_1.tif
32636       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_12MAR2018.tif
32639       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif
32653       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_25DEC2015.tif
                                              ...                                      
193498            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_19NOV1972_mosaic.jp2
193504            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2
193507            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_10NOV1977_mosaic.tif
193508            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif
193511            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2
Name: 0, Length: 2704, dtype: ob

In [6]:
Gabrielle_images = filelist[filelist.str.startswith("Gabrielle") & filelist.str.endswith((".jpg", ".jp2", ".tif"))]

In [7]:
# When fuzzy matching, ignore these strings
# _0 will ignore leading zeros in dates
strings_to_delete = ["_mosaic", "_mosiac", "_mosaid", ".mosaic", "_cliff", "_beach", "_beachcliffsegment", "_MF.shp", "_MT.shp", "_0", "_1.tif", "_2.tif", "_3.tif", "_LDS", "_", " "]

def fuzz_preprocess(filename):
    for s in strings_to_delete:
        filename = filename.replace(s, "")
    # Case-insensitive
    filename = filename.lower()
    # Ignore extension
    filename = os.path.splitext(filename)[0]
    # Basename only
    filename = os.path.basename(filename)
    return filename

def get_matching_image(filename):
    if filename.startswith("Gabrielle"):
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=Gabrielle_images, processor=fuzz_preprocess)
        return match, score
    dirname = os.path.dirname(filename)
    dirname = os.path.dirname(dirname)
    RL_dirname = dirname.replace("MaxarImagery/HighFreq", "Retrolens")
    Maxar_dirname = dirname.replace("Retrolens", "MaxarImagery/HighFreq")
    all_files_in_folder = images[images.str.startswith((RL_dirname, Maxar_dirname))]
    if len(all_files_in_folder) == 0:
        return "", 0
    match, score, index = rapidfuzz.process.extractOne(query=filename, choices=all_files_in_folder, processor=fuzz_preprocess)
    return match, score

df["matched_image"], df["match_score"] = zip(*df.filename.apply(get_matching_image))
print("Perfect matches:", sum(df.match_score == 100))
print("Imperfect matches:", sum(df.match_score < 100))

Perfect matches: 2025
Imperfect matches: 560


In [8]:
def get_shapefile_meta(filename):
    source = "Unknown"
    try:
        shapefile = gpd.read_file(prefix+filename)
    except:
        print(f"Can't read {filename}")
        return source, 0
    if "Source" not in shapefile.columns:
        if filename.startswith("Retrolens"):
            source = "RL"
        elif filename.startswith("MaxarImagery/HighFreq"):
            source = "MAX"
    else:
        sources = shapefile.Source.unique()
        if len(sources) == 0 or not sources[0]:
            #print(f"{filename} has no sources")
            if filename.startswith("MaxarImagery/HighFreq"):
                source = "MAX"
        elif len(sources) == 1:
            source = sources[0]
        elif len(sources) > 1:
            print(f"{filename} has ambiguous sources: {sources}")
            source = sources[0]
    n_lines = len(shapefile.dropna(subset="geometry").explode(index_parts=False))
    return source, n_lines

df["Source"], df["n_lines"] = zip(*process_map(get_shapefile_meta, df.filename))
df.Source.value_counts(dropna=False)

/tmp/ipykernel_2388344/4181019647.py:27: TqdmWarning: Iterable length 2585 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  df["Source"], df["n_lines"] = zip(*process_map(get_shapefile_meta, df.filename))


  0%|          | 0/2585 [00:00<?, ?it/s]

MaxarImagery/HighFreq/Northland/MarsdenPoint/Shorelines/MarsdenPoint_01JAN2020.shp has ambiguous sources: ['MAX' None]
MaxarImagery/HighFreq/Northland/Aranga/Shorelines/Aranga_09NOV2012.shp has ambiguous sources: ['MAX' None]
MaxarImagery/HighFreq/Wellington/KapitiMid/Shorelines/KapitiMid_31OCT2019.shp has ambiguous sources: ['MAX' None]
Retrolens/HawkesBay/Clifton_Haumoana/Shorelines/Clifton_Haumoana_23SEP1963.shp has ambiguous sources: ['RL' None]
Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_06MAR2019.shp has ambiguous sources: ['LDS' None]
Retrolens/Southland/HaldaneBay/Shorelines/HaldaneBay_07OCT1985.shp has ambiguous sources: ['RL' None]
Retrolens/Taranaki/OpunakeBeach/Shorelines/OpunakeBeach_11FEB2017.shp has ambiguous sources: ['LDS' None]
Retrolens/Taranaki/South Taranaki/OpunakeBeach/Shorelines/OpunakeBeach_11FEB2017.shp has ambiguous sources: ['LDS' None]
Retrolens/Auckland/PakiriBeach/Shorelines/PakiriBeach_24OCT1953.shp has ambiguous sources: ['RL' None]
Retrolens/Tasman/

Source
RL         1141
MAX         640
LDS         415
RLN         165
Unknown      66
PLE          45
Max          41
CRI          24
NEO          18
RLS          12
JIL           4
max           3
RS            2
PNE           2
GE1           2
JIN           1
VEX           1
WV3           1
SAT           1
Rl            1
Name: count, dtype: int64

In [9]:
df.sort_values(by="match_score")

,filename,matched_image,match_score,Source,n_lines
113442,Retrolens/Waikato/AwakinoNorth/Shorelines/AwakinoNorth_15FEB2022.shp,,0.0,LDS,1
111915,Retrolens/Waikato/TePuru/Shorelines/TePuru_16FEB2018.shp,,0.0,LDS,2
112034,Retrolens/Waikato/Awakino/Shorelines/Awakino_10MAR2018.shp,,0.0,LDS,4
112039,Retrolens/Waikato/Awakino/Shorelines/Awakino_15FEB2022.shp,,0.0,LDS,5
112225,Retrolens/Waikato/KaawaStream_North/Shorelines/KaawaStream_North_15FEB2022.shp,,0.0,LDS,5
...,...,...,...,...,...
83620,Retrolens/Nelson/Nelson/Shorelines/Nelson_06FEB1967_mosaic.shp,Retrolens/Nelson/Nelson/Stack/Nelson_06FEB1967_mosaic.jp2,100.0,RL,5
83625,Retrolens/Nelson/Nelson/Shorelines/Nelson_11JAN1981_mosaic.shp,Retrolens/Nelson/Nelson/Stack/Nelson_11JAN1981_mosaic.jp2,100.0,RL,8
83628,Retrolens/Nelson/Nelson/Shorelines/Nelson_13SEP1985_mosaic.shp,Retrolens/Nelson/Nelson/Stack/Nelson_13SEP1985_mosaic.jp2,100.0,RL,9
83583,Retrolens/Nelson/Nelson/Shorelines/Nelson_14MAR1948_mosaic.shp,Retrolens/Nelson/Nelson/Stack/Nelson_14MAR1948_mosaic.jp2,100.0,RL,9


In [10]:
df.sort_values(by="match_score").to_csv(prefix+"Nick/shoreline_image_matching.csv", index=False)

In [22]:
df[df.filename.str.contains("Critchlow")]

,filename,matched_image,match_score,Source,n_lines
49088,MaxarImagery/HighFreq/Taranaki/Critchlow/NewPlymouth/Shorelines/NewPlymouth_09FEB2011_beach.shp,,0.0,CRI,10
49093,MaxarImagery/HighFreq/Taranaki/Critchlow/NewPlymouth/Shorelines/NewPlymouth_11MAR2011_beach.shp,,0.0,CRI,1
49102,MaxarImagery/HighFreq/Taranaki/Critchlow/NewPlymouth/Shorelines/NewPlymouth_11MAR2011_cliff.shp,,0.0,CRI,12
49181,MaxarImagery/HighFreq/Taranaki/Critchlow/OakuraSouth/Shorelines/Oakura_South_16JAN2012.shp,,0.0,CRI,6
49189,MaxarImagery/HighFreq/Taranaki/Critchlow/OakuraSouth/Shorelines/Oakura_South_11MAR2011.shp,,0.0,CRI,4
49215,MaxarImagery/HighFreq/Taranaki/Critchlow/MohakatinoRiver_South/Shorelines/MohakatinoRiver_18APR2011.shp,,0.0,CRI,29
49263,MaxarImagery/HighFreq/Taranaki/Critchlow/Oakura/Shorelines/Oakura_11MAR2011_cliff.shp,,0.0,CRI,4
49270,MaxarImagery/HighFreq/Taranaki/Critchlow/Oakura/Shorelines/Oakura_11MAR2011_beach.shp,,0.0,CRI,5


### For those perfect matches, get the metadata from the corresponding matched image

In [11]:
# Investigate metadata about the matched images

def get_meta(tup):
    i, row = tup
    try:
        image = rio.open(prefix + row.matched_image)
    except:
        print(f"Can't read {row.matched_image}")
        return
    try:
        gdf = gpd.read_file(prefix + row.filename)
        row = row.to_dict()
        row["n_lines"] = len(gdf.dropna(subset="geometry").explode(index_parts=False))
    except: 
        print(f"Can't read {row['filename']}")
        return
    
    row.update(image.profile)
    row["GCPs"] = len(image.gcps[0])
    row["res"] = image.res
    row["CPS"] = "CPS" in gdf.columns
    return row

metafile = "meta.csv"
if use_cache and os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(thread_map(get_meta, df[df.match_score == 100].iterrows(), total=sum(df.match_score == 100)))
    meta.to_csv(metafile, index=False)
meta

,filename,matched_image,match_score,Source,n_lines,driver,dtype,nodata,width,height,count,crs,transform,blockxsize,blockysize,tiled,interleave,GCPs,res,CPS,compress,photometric
0,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_18FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Waikato/Matarangi and surrounds/Matarangi_18FEB2023.tif,100.0,PLE,1,GTiff,uint32,6.553600e+04,36122,21669,3,NaN,"| 0.30, 0.00, 1832041.81|\n| 0.00,-0.30, 5936761.33|\n| 0.00, 0.00, 1.00|",128.0,128,True,pixel,6,"(0.3000000000000026, 0.3000000000000086)",True,NaN,NaN
1,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_24DEC2022.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,100.0,PLE,1,GTiff,uint16,6.553500e+04,13788,15017,4,NaN,"| 0.50, 0.00, 1832772.89|\n| 0.00,-0.50, 5936498.44|\n| 0.00, 0.00, 1.00|",128.0,128,True,pixel,7,"(0.5, 0.5)",True,NaN,NaN
2,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/NewChums_18FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Waikato/Matarangi and surrounds/NewChums_18FEB2023.tif,100.0,PLE,1,GTiff,uint32,6.553600e+04,36122,21669,3,NaN,"| 0.30, 0.00, 1832041.81|\n| 0.00,-0.30, 5936761.33|\n| 0.00, 0.00, 1.00|",128.0,128,True,pixel,6,"(0.3000000000000026, 0.3000000000000086)",True,NaN,NaN
3,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_24DEC2022.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Whangapoua_24DEC2022.tif,100.0,PLE,1,GTiff,uint16,6.553500e+04,13788,15017,4,NaN,"| 0.50, 0.00, 1832772.89|\n| 0.00,-0.50, 5936498.44|\n| 0.00, 0.00, 1.00|",128.0,128,True,pixel,7,"(0.5, 0.5)",True,NaN,NaN
4,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_18FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Waikato/Matarangi and surrounds/Whangapoua_18FEB2023.tif,100.0,PLE,2,GTiff,uint32,6.553600e+04,36122,21669,3,NaN,"| 0.30, 0.00, 1832041.81|\n| 0.00,-0.30, 5936761.33|\n| 0.00, 0.00, 1.00|",128.0,128,True,pixel,6,"(0.3000000000000026, 0.3000000000000086)",True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_17APR1952.shp,Retrolens/Wellington/KapitiSouth/Stack/KapitiSouth_17APR1952_mosaic.tif,100.0,RL,5,GTiff,int32,2.147484e+09,14167,31280,1,NaN,"| 0.27, 0.00, 1763537.25|\n| 0.00,-0.27, 5469157.98|\n| 0.00, 0.00, 1.00|",128.0,128,True,band,7,"(0.26838646488296874, 0.2683864648829636)",True,NaN,NaN
2021,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp,Retrolens/Wellington/KapitiSouth/Stack/KapitiSouth_02JAN1988_mosaic.jp2,100.0,RL,9,JP2OpenJPEG,uint16,2.560000e+02,10654,18819,1,NaN,"| 0.39, 0.00, 1762602.58|\n| 0.00,-0.39, 5466346.26|\n| 0.00, 0.00, 1.00|",1024.0,1024,True,NaN,4,"(0.3894503758518079, 0.3894503758518002)",True,NaN,NaN
2022,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp,Retrolens/Wellington/KapitiSouth/Stack/KapitiSouth_06OCT1980_mosaic.tif,100.0,RL,11,GTiff,int32,2.147484e+09,12620,29841,1,NaN,"| 0.36, 0.00, 1762518.60|\n| 0.00,-0.36, 5469658.24|\n| 0.00, 0.00, 1.00|",128.0,128,True,band,10,"(0.35964806541597927, 0.3596480654159714)",True,NaN,NaN
2023,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp,Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif,100.0,RLN,6,GTiff,uint16,2.560000e+02,40393,28128,1,"PROJCS[""NZGD2000 / New Zealand Transverse Mercator 2000"",GEOGCS[""NZGD2000"",DATUM[""New_Zealand_Geodetic_Datum_2000"",SPHEROID[""GRS 1980"",6378137,298.257222101004,AUTHORITY[""EPSG"",""7019""]],AUTHORITY[...","| 0.13, 0.00, 1757756.38|\n| 0.00,-0.13, 5459482.04|\n| 0.00, 0.00, 1.00|",128.0,128,True,band,0,"(0.1284827509237925, 0.12848275092379952)",True,NaN,NaN


## Read LDS index tiles

In [12]:
index_tiles = filelist[filelist.str.contains(".+index-tiles.+.shp$")]
index_tiles

4515      Gabrielle/Imagery/post_storm/LINZ/HawkesBay/hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index-tiles-Copy.shp
10077             Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp
11954          Gabrielle/Imagery/post_storm/LINZ/Gisborne/gisborne-02m-cyclone-gabrielle-aerial-photos-index-tiles-202.shp
13935          Gabrielle/Imagery/pre-storm/Waikato/TairuaPauanui/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
14054               Gabrielle/Imagery/pre-storm/Waikato/LINZtemp/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
                                                                ...                                                       
229234             SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-03m-rural-aerial-photos-index-tiles-2019.shp
229239        SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2018-2019.shp
236844          

In [13]:
def read_index_tile(f):
    gdf = gpd.read_file(prefix+f).to_crs(2193)
    gdf["filename"] = f
    return gdf

index_tiles = pd.concat(process_map(read_index_tile, index_tiles))
len(index_tiles)

  0%|          | 0/79 [00:00<?, ?it/s]

147407

In [14]:
index_tiles.Captured = index_tiles.Captured.replace({"05, 06, 09, 10 Feb. 2014": "2014-02-05", "11 & 26 March 2014": "2014-03-11"})
index_tiles["parsed_date"] = pd.NA
for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured"]:
    index_tiles["parsed_date"].update(pd.to_datetime(index_tiles[col].str.split(",|-|to").str[0], dayfirst=True, format="mixed", errors="ignore"))
index_tiles.parsed_date = index_tiles.parsed_date.astype("string")
index_tiles.parsed_date.value_counts(dropna=False)

/home/nyou045/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


parsed_date
2017                   23649
<NA>                   22679
2022                   11348
2023                    3607
24/02/2017              2685
                       ...  
9/11/2016                  1
2029-03-12 13:01:12        1
2005-05-12 12:05:22        1
2016-11-15 15:12:28        1
2028-11-15 16:03:27        1
Name: count, Length: 384, dtype: Int64

## LDS index tile matching

In [15]:
LDS = df[(df.Source == "LDS") & (df.match_score < 100)].sort_values("match_score")
LDS

,filename,matched_image,match_score,Source,n_lines
112696,Retrolens/Waikato/UrawhitikiPoint/Shorelines/UrawhitikiPoint_15FEB2022.shp,,0.000000,LDS,1
110041,Retrolens/Waikato/WaikawauRiver/Shorelines/WaikawauRiver_15FEB2022.shp,,0.000000,LDS,3
112760,Retrolens/Waikato/ColvilleBay/Shorelines/ColvilleBay_16FEB2018.shp,,0.000000,LDS,6
112701,Retrolens/Waikato/UrawhitikiPoint/Shorelines/UrawhitikiPoint_10MAR2018.shp,,0.000000,LDS,1
112593,Retrolens/Waikato/KennedyBay/Shorelines/KennedyBay_16FEB2018.shp,,0.000000,LDS,2
...,...,...,...,...,...
160436,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_01APR2006.shp,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Stack/Ryans_Pipikaretu_Penguin_TeRauoneBeach_17MAR2000_mosaic.jp2,94.252874,LDS,8
128159,Retrolens/Taranaki/Oakura/Shorelines/Oakura_31OCT2016_cliff.shp,MaxarImagery/HighFreq/Taranaki/Oakura/Imagery/Stack/Oakura_03OCT2016.tif,96.551724,LDS,5
128111,Retrolens/Taranaki/Oakura/Shorelines/Oakura_31OCT2016_beach.shp,MaxarImagery/HighFreq/Taranaki/Oakura/Imagery/Stack/Oakura_03OCT2016.tif,96.551724,LDS,3
150632,Retrolens/Canterbury/GoreBay/Shorelines/GoreBay_8FEB2022.shp,Retrolens/Canterbury/GoreBay/Stack/GoreBay_LDS_18FEB2022.TIF,96.774194,LDS,5


In [16]:
def get_resolution(filename):
  gdf = gpd.read_file(prefix+filename)
  if "LDS" not in gdf.Source.unique():
    return {"filename": filename}
  bounds = gdf.total_bounds
  intersecting_tiles = index_tiles[index_tiles.intersects(box(*bounds))]
  if filename.startswith("Gabrielle"):
    intersecting_tiles = intersecting_tiles[intersecting_tiles.filename.str.startswith("Gabrielle")]
  if len(intersecting_tiles) == 0:
    print(f"{filename} doesn't intersect any index tiles")
    return {"filename": filename}
  if "Date" not in gdf.columns:
    #print(f"{filename} missing Date column")
    date = gdf.DSASdate.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  elif "DSASDate" not in gdf.columns:
    #print(f"{filename} missing DSASDate column")
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  else:
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASDate.unique()[0]
  if not DSASdate:
    DSASdate = date

  date_options = []
  date_to_col = {}
  for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured"]:#, "parsed_date"]:
    options_for_col = intersecting_tiles[col].dropna().unique().tolist()
    date_options.extend(options_for_col)
    for date_option in options_for_col:
      date_to_col[date_option] = col
  display(date, DSASdate, gdf, intersecting_tiles.dropna(axis=1, how="all"), date_options, date_to_col)
  if not date_options:
    if "hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index" in intersecting_tiles.filename.unique()[0]:
      return {
        "filename": filename,
        "matched_index_tiles": intersecting_tiles.filename.unique(),
        "date": date,
        "matched_date": "SPECIAL_OVERRIDE",
        "match_score": 100,
        "Pixel_ER": .1
      }
    else:
      print(f"No date options in {intersecting_tiles.filename.unique()}")
      return {"filename": filename}
  else:
    match = False
    if DSASdate in date_options:
      match = DSASdate
      score = 100
    for option in date_options:
      if DSASdate in option or date in option:
        match = option
        score = 100
    if not match:
      match, score, index = rapidfuzz.process.extractOne(query=date, choices=date_options, processor=lambda s: s.replace("-", ""))
    col_for_match = date_to_col[match]
    tiles_from_this_date = intersecting_tiles[intersecting_tiles[col_for_match] == match]
    GSDM = []
    for col in ['GSDM', 'ORTHO_GSD', 'Ortho_GSD', 'GSDm', 'GSD', 'GSD_M', 'GSD_CM', 'gsdM']:
      GSDM.extend(tiles_from_this_date[col].dropna().astype(str).str.strip("m").unique())
    if len(GSDM) == 0:
      tilefile = tiles_from_this_date.filename.unique()[0]
      if "-04m" in tilefile:
        GSDM = .4
      elif "-0075m" in tilefile:
        GSDM = .075
    elif len(GSDM) == 1:
      GSDM = GSDM[0]
    elif len(GSDM) > 1:
      #print(f"Ambiguous GSDM: {GSDM}")
      GSDM = GSDM[0]
    return {
      "filename": filename,
      "matched_index_tiles": tiles_from_this_date.filename.unique().tolist(),
      "date": date,
      "DSASDate": DSASdate,
      "matched_date": match,
      "matched_date_col": col_for_match,
      "match_score": score,
      "Pixel_ER": GSDM
    }

#get_resolution("Retrolens/Taranaki/PariokariwaPoint/Shorelines/PariokariwaPoint_19MAR2017_Taranaki399_cliff.shp")
LDS = pd.DataFrame(process_map(get_resolution, LDS.filename)).sort_values("match_score")
print("Perfect matches:", sum(LDS.match_score == 100))
print("Imperfect matches:", sum(LDS.match_score < 100))
LDS

  0%|          | 0/397 [00:00<?, ?it/s]

Perfect matches: 282
Imperfect matches: 115


,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
65,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,30.000000,0.3
205,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
395,Retrolens/Canterbury/GoreBay/Shorelines/GoreBay_8FEB2022.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2022-02-08,08/02/2022,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,37.500000,0.3
165,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
345,Retrolens/Canterbury/WainononLagoon_PareoaRiver/Shorelines/WainonoLagoon_PareoraRiver_02DEC2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-12-02,02/12/2017,02/12/17 to 08/02/18,FLOWN,42.750000,0.3
...,...,...,...,...,...,...,...,...
139,Gabrielle/Shorelines/Gisborne/Whareonga/Whareonga_16FEB2022.shp,"[Gabrielle/Imagery/pre-storm/LINZ/Gisborne/Imagery/gisborne-02m-rural-aerial-photos-index-tiles-2021-2023Copy.shp, Gabrielle/Imagery/pre-storm/LINZ/Gisborne/Imagery/gisborne-02m-rural-aerial-photo...",2022-02-16,16/02/2022,16/02/2022,FLOWN,100.000000,0.2
138,Retrolens/Gisborne/AnauraBay/Shorelines/AnauraBay_15FEB2018.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-gisborne-03m-rural-aerial-photos-index-tiles-2017-2019-SHP (1)/gisborne-03m-rural-aerial-photos-index-tiles-2017-2019.shp],2018-02-15,15/02/2018,15/02/2018,FLOWN,100.000000,0.3
137,Gabrielle/Shorelines/Gisborne/Wainui/Tatapouri_07MAR2023.shp,[Gabrielle/Imagery/post_storm/LINZ/Gisborne/gisborne-02m-cyclone-gabrielle-aerial-photos-index-tiles-202.shp],2023-03-07,07/03/2023,07/03/2023,FLOWN,100.000000,0.2
135,Retrolens/Northland/WhareanaBay/Shorelines/WhareanaBay_07NOV2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Northland Footrpints/northland-04m-rural-aerial-photos-index-tiles-2014-2016.shp],2015-11-07,07/11/2015,07/11/2015,DATE_DMY,100.000000,0.4


In [17]:
with pd.option_context("display.max_rows", None):
  display(LDS[LDS.match_score < 100])

,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
65,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,30.000000,0.3
205,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
395,Retrolens/Canterbury/GoreBay/Shorelines/GoreBay_8FEB2022.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2022-02-08,08/02/2022,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,37.500000,0.3
165,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
345,Retrolens/Canterbury/WainononLagoon_PareoaRiver/Shorelines/WainonoLagoon_PareoraRiver_02DEC2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-12-02,02/12/2017,02/12/17 to 08/02/18,FLOWN,42.750000,0.3
38,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,[Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp],05/04/2023,05/04/2023,2023-04-05,FLOWN,44.444444,0.10
178,Retrolens/HawkesBay/OceanBeach/Shorelines/OceanBeach_06MAR2019.shp,[SpatialData/Mosaics/Footprint shapefiles/Hawkes Bay Footprints/hawkes-bay-03m-rural-aerial-photos-index-tiles-2019-2020.shp],2019-03-06,6/03/2019,05/03/2019,FLOWN,44.444444,0.3
317,Retrolens/Otago/Warrington/Shorelines/Warrington_30MAR2006.shp,[SpatialData/Mosaics/Footprint shapefiles/Otago footprints/lds-dunedin-01m-urban-aerial-photos-index-tiles-2018-2019-SHP/dunedin-01m-urban-aerial-photos-index-tiles-2018-2019.shp],1899-12-30,1899-12-30,"07/04/2019,08/02/2019,06/11/2018,14/01/18",FLOWN,45.000000,0.1
287,Retrolens/Taranaki/NewPlymouth_Airport/Shorelines/NewPlymouth Airport_19MAR2017.shp,[SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-03m-rural-aerial-photos-index-tiles-2016-2018.shp],2017-03-19,19/03/2017,19/3/2017,FLOWN,47.058824,0.3
362,Retrolens/Taranaki/South Taranaki/Hawera_WaihiBeach/Shorelines/Hawera_WaihiBeach_19MAR2017.shp,[SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-03m-rural-aerial-photos-index-tiles-2016-2018.shp],2017-03-19,19/03/2017,19/3/2017,FLOWN,47.058824,0.3


In [18]:
# Stop Excel interpreting dates
for col in ["date", "DSASDate", "matched_date"]:
    LDS[col] = "'" + LDS[col]
LDS.to_csv(prefix + "Nick/LDS_matches.csv", index=False)